In [1]:
import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import database
import pandas as pd
from snowballing.operations import reload, load_work_map_all_years
reload()
from snowballing.strategies import Strategy
strategy = Strategy({})

def pages(work):
    pp = getattr(work, 'pp', '')
    if not pp:
        return 0
    split = list(map(int, pp.split('--')))
    if len(split) == 1:
        return split[0]
    return split[-1] - split[0] + 1

In [2]:
selected = sorted({
    w
    for _, w in load_work_map_all_years()
    if w.place.acronym == "IPAW"
    if pages(w) >= 6
    if w.year >= 2008
}, key=lambda w: int(w.metakey[-5:-1]))

In [3]:
len(selected)

98

In [4]:
places = {
    w.place
    for _, w in load_work_map_all_years()
}

In [5]:
len(places)

341

In [6]:
{
    p.type for p in places
}

{'Archive',
 'Book',
 'Conference',
 'Journal',
 'Lang',
 'Magazine',
 'Patent',
 'Peer Reviewed Book',
 'Tech Report',
 'Thesis'}

In [7]:
result = []

for w in selected:
    scholar = int(getattr(w, "scholar_count", 0))
    actual = len(set(strategy.rev_ref[w]))
    books = len({r for r in strategy.rev_ref[w]
                 if r.place.type == "Book"})
    thesis = len({r for r in strategy.rev_ref[w]
                  if r.place.type == "Thesis"})
    lang = len({r for r in strategy.rev_ref[w]
                if r.place.type == "Lang"})
    other = len({r for r in strategy.rev_ref[w]
                 if r.place.type in (
                    "Archive", "Magazine", "Patent",
                    "Tech Report"
                 )})
    result.append([
        w, scholar, actual, books, thesis, lang, other,
        actual - books,
        actual - thesis - books,
        actual - thesis - books - lang - other,
    ])
    
header = [
    "ID", "Scholar", "Without Repetition", 
    "Books", "Thesis/Dissertation/Monograph (TDM)",
    "Unindentified Languages",
    "Other Grey Literature",
    "Without Books",
    "Without Books and TDM",
    "Only Conference and Journal papers"
]

metakey = [[row[0].metakey] + row[1:] for row in result]
df = pd.DataFrame(metakey, columns=header)
df

,ID,Scholar,Without Repetition,Books,Thesis/Dissertation/Monograph (TDM),Unindentified Languages,Other Grey Literature,Without Books,Without Books and TDM,Only Conference and Journal papers
0,callahan2008a,27,27,2,3,0,1,25,22,21
1,mackenzie2008a,15,15,2,0,0,1,13,13,12
2,oliveira2008a,34,33,0,7,1,2,33,26,23
3,bowers2008a,77,75,6,11,6,5,69,58,47
4,kwasnikowska2008a,24,24,0,6,2,1,24,18,15
5,frew2008a,17,17,1,2,0,4,16,14,10
6,bochner2008a,17,16,1,5,0,1,15,10,9
7,chen2008a,6,6,1,0,1,0,5,5,4
8,nagappan2008a,9,9,0,1,0,0,9,8,8
9,missier2008b,11,9,0,2,0,1,9,7,6


In [8]:
def hindex(result, index, category):
    sort_by_index = [
        (w[index], w[0].year, w[0].name, w[0].authors)
        for w in sorted(result, reverse=True, key=lambda x: x[index])
    ]
    hindex_list = [
        x for i, x in enumerate(sort_by_index)
        if x[0] >= i + 1
    ]
    print(f"{category}: {len(hindex_list)}")
    df = pd.DataFrame(sort_by_index, columns=["Citations", "Year", "Name", "Authors"])
    df.index += 1 
    return df

In [9]:
category = "Scholar"
index = 1
df = hindex(result, index, category)
df.to_excel("scholar.xlsx")
df.style.apply(
    lambda s: ['background-color: #FF8888' if s.name > s.Citations else "" for _ in s], axis=1
)

Scholar: 20


,Citations,Year,Name,Authors
1,119,2010,Publishing and consuming provenance metadata on the web of linked data,"Hartig, Olaf and Zhao, Jun"
2,77,2008,Kepler/pPOD: Scientific workflow and provenance support for assembling the tree of life,"Bowers, Shawn and Mcphillips, Timothy and Riddle, Sean and Anand, Manish Kumar and Ludäscher, Bertram"
3,76,2008,Data lineage model for Taverna workflows with lightweight annotation requirements,"Missier, Paolo and Belhajjame, Khalid and Zhao, Jun and Roos, Marco and Goble, Carole"
4,73,2010,Janus: from workflows to semantic provenance and linked open data,"Missier, Paolo and Sahoo, Satya S and Zhao, Jun and Goble, Carole and Sheth, Amit"
5,67,2014,noWorkflow: capturing and analyzing provenance of scripts,"Murta, Leonardo and Braganholo, Vanessa and Chirigati, Fernando and Koop, David and Freire, Juliana"
6,64,2008,A provenance-based fault tolerance mechanism for scientific workflows,"Crawl, Daniel and Altintas, Ilkay"
7,41,2008,A first study on clustering collections of workflow graphs,"Santos, Emanuele and Lins, Lauro and Ahrens, James P and Freire, Juliana and Silva, Claudio T"
8,40,2008,Provenance in sensornet republishing,"Park, Unkyu and Heidemann, John"
9,37,2008,Advances and challenges for scalable provenance in stream processing systems,"Misra, Archan and Blount, Marion and Kementsietsidis, Anastasios and Sow, Daby and Wang, Min"
10,34,2008,Using provenance to improve workflow design,"De Oliveira, Frederico T and Murta, Leonardo and Werner, Claudia and Mattoso, Marta"


In [10]:
category = "Without Repetitions"
index = 2
df = hindex(result, index, category)
df.style.apply(
    lambda s: ['background-color: #FF8888' if s.name > s.Citations else "" for _ in s], axis=1
)

Without Repetitions: 19


,Citations,Year,Name,Authors
1,116,2010,Publishing and consuming provenance metadata on the web of linked data,"Hartig, Olaf and Zhao, Jun"
2,75,2008,Kepler/pPOD: Scientific workflow and provenance support for assembling the tree of life,"Bowers, Shawn and Mcphillips, Timothy and Riddle, Sean and Anand, Manish Kumar and Ludäscher, Bertram"
3,73,2008,Data lineage model for Taverna workflows with lightweight annotation requirements,"Missier, Paolo and Belhajjame, Khalid and Zhao, Jun and Roos, Marco and Goble, Carole"
4,69,2010,Janus: from workflows to semantic provenance and linked open data,"Missier, Paolo and Sahoo, Satya S and Zhao, Jun and Goble, Carole and Sheth, Amit"
5,66,2014,noWorkflow: capturing and analyzing provenance of scripts,"Murta, Leonardo and Braganholo, Vanessa and Chirigati, Fernando and Koop, David and Freire, Juliana"
6,62,2008,A provenance-based fault tolerance mechanism for scientific workflows,"Crawl, Daniel and Altintas, Ilkay"
7,40,2008,A first study on clustering collections of workflow graphs,"Santos, Emanuele and Lins, Lauro and Ahrens, James P and Freire, Juliana and Silva, Claudio T"
8,40,2008,Provenance in sensornet republishing,"Park, Unkyu and Heidemann, John"
9,36,2008,Advances and challenges for scalable provenance in stream processing systems,"Misra, Archan and Blount, Marion and Kementsietsidis, Anastasios and Sow, Daby and Wang, Min"
10,33,2008,Using provenance to improve workflow design,"De Oliveira, Frederico T and Murta, Leonardo and Werner, Claudia and Mattoso, Marta"


In [11]:
category = "Without Books"
index = 7
df = hindex(result, index, category)
df.style.apply(
    lambda s: ['background-color: #FF8888' if s.name > s.Citations else "" for _ in s], axis=1
)

Without Books: 19


,Citations,Year,Name,Authors
1,108,2010,Publishing and consuming provenance metadata on the web of linked data,"Hartig, Olaf and Zhao, Jun"
2,69,2008,Kepler/pPOD: Scientific workflow and provenance support for assembling the tree of life,"Bowers, Shawn and Mcphillips, Timothy and Riddle, Sean and Anand, Manish Kumar and Ludäscher, Bertram"
3,67,2008,Data lineage model for Taverna workflows with lightweight annotation requirements,"Missier, Paolo and Belhajjame, Khalid and Zhao, Jun and Roos, Marco and Goble, Carole"
4,65,2010,Janus: from workflows to semantic provenance and linked open data,"Missier, Paolo and Sahoo, Satya S and Zhao, Jun and Goble, Carole and Sheth, Amit"
5,60,2014,noWorkflow: capturing and analyzing provenance of scripts,"Murta, Leonardo and Braganholo, Vanessa and Chirigati, Fernando and Koop, David and Freire, Juliana"
6,58,2008,A provenance-based fault tolerance mechanism for scientific workflows,"Crawl, Daniel and Altintas, Ilkay"
7,39,2008,A first study on clustering collections of workflow graphs,"Santos, Emanuele and Lins, Lauro and Ahrens, James P and Freire, Juliana and Silva, Claudio T"
8,37,2008,Provenance in sensornet republishing,"Park, Unkyu and Heidemann, John"
9,34,2008,Advances and challenges for scalable provenance in stream processing systems,"Misra, Archan and Blount, Marion and Kementsietsidis, Anastasios and Sow, Daby and Wang, Min"
10,33,2008,Using provenance to improve workflow design,"De Oliveira, Frederico T and Murta, Leonardo and Werner, Claudia and Mattoso, Marta"


In [12]:
category = "Without Books, Theses, Dissertations, Monographs"
index = 8
df = hindex(result, index, category)
df.style.apply(
    lambda s: ['background-color: #FF8888' if s.name > s.Citations else "" for _ in s], axis=1
)

Without Books, Theses, Dissertations, Monographs: 18


,Citations,Year,Name,Authors
1,90,2010,Publishing and consuming provenance metadata on the web of linked data,"Hartig, Olaf and Zhao, Jun"
2,60,2008,Data lineage model for Taverna workflows with lightweight annotation requirements,"Missier, Paolo and Belhajjame, Khalid and Zhao, Jun and Roos, Marco and Goble, Carole"
3,58,2008,Kepler/pPOD: Scientific workflow and provenance support for assembling the tree of life,"Bowers, Shawn and Mcphillips, Timothy and Riddle, Sean and Anand, Manish Kumar and Ludäscher, Bertram"
4,56,2010,Janus: from workflows to semantic provenance and linked open data,"Missier, Paolo and Sahoo, Satya S and Zhao, Jun and Goble, Carole and Sheth, Amit"
5,50,2014,noWorkflow: capturing and analyzing provenance of scripts,"Murta, Leonardo and Braganholo, Vanessa and Chirigati, Fernando and Koop, David and Freire, Juliana"
6,47,2008,A provenance-based fault tolerance mechanism for scientific workflows,"Crawl, Daniel and Altintas, Ilkay"
7,33,2008,Provenance in sensornet republishing,"Park, Unkyu and Heidemann, John"
8,31,2008,A first study on clustering collections of workflow graphs,"Santos, Emanuele and Lins, Lauro and Ahrens, James P and Freire, Juliana and Silva, Claudio T"
9,29,2008,Advances and challenges for scalable provenance in stream processing systems,"Misra, Archan and Blount, Marion and Kementsietsidis, Anastasios and Sow, Daby and Wang, Min"
10,26,2008,Using provenance to improve workflow design,"De Oliveira, Frederico T and Murta, Leonardo and Werner, Claudia and Mattoso, Marta"


In [13]:
category = "Without Grey literature:"
index = 9
df = hindex(result, index, category)
df.style.apply(
    lambda s: ['background-color: #FF8888' if s.name > s.Citations else "" for _ in s], axis=1
)

Without Grey literature:: 17


,Citations,Year,Name,Authors
1,79,2010,Publishing and consuming provenance metadata on the web of linked data,"Hartig, Olaf and Zhao, Jun"
2,54,2008,Data lineage model for Taverna workflows with lightweight annotation requirements,"Missier, Paolo and Belhajjame, Khalid and Zhao, Jun and Roos, Marco and Goble, Carole"
3,47,2008,Kepler/pPOD: Scientific workflow and provenance support for assembling the tree of life,"Bowers, Shawn and Mcphillips, Timothy and Riddle, Sean and Anand, Manish Kumar and Ludäscher, Bertram"
4,47,2010,Janus: from workflows to semantic provenance and linked open data,"Missier, Paolo and Sahoo, Satya S and Zhao, Jun and Goble, Carole and Sheth, Amit"
5,42,2008,A provenance-based fault tolerance mechanism for scientific workflows,"Crawl, Daniel and Altintas, Ilkay"
6,42,2014,noWorkflow: capturing and analyzing provenance of scripts,"Murta, Leonardo and Braganholo, Vanessa and Chirigati, Fernando and Koop, David and Freire, Juliana"
7,29,2008,A first study on clustering collections of workflow graphs,"Santos, Emanuele and Lins, Lauro and Ahrens, James P and Freire, Juliana and Silva, Claudio T"
8,28,2008,Provenance in sensornet republishing,"Park, Unkyu and Heidemann, John"
9,24,2010,Understanding collaborative studies through interoperable workflow provenance,"Altintas, Ilkay and Anand, Manish Kumar and Crawl, Daniel and Bowers, Shawn and Belloum, Adam and Missier, Paolo and Ludäscher, Bertram and Goble, Carole A and Sloot, Peter MA"
10,23,2008,Using provenance to improve workflow design,"De Oliveira, Frederico T and Murta, Leonardo and Werner, Claudia and Mattoso, Marta"
